In [1]:
!pip install langchain_openai
!pip install langchain
!pip install langchain-community
!pip install faiss-gpu




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 828.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-4B7ECmBE3UztEXMsezhvT3BlbkFJwrVcvP5C1aiuH5Ipg6oC"
os.environ['LANGCHAIN_ENDPOINT']= "https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']= "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"
os.environ['LANGCHAIN_TRACING_V2']='true'


In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


In [5]:
from langchain.agents import tool


In [6]:

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert in the sport of Skateboarding. You have to answer these questions to a very high level of accuracy. Answer these questions in as much detail as possible and provide all the information you know.You are not restricted to answering only from the document you will be provided. You can augment additonal knowledge if and when necessary to provide as much detail as possible. If the question is not related to skateboarding, ask the user to ask only related questions.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [7]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("/content/SkateboardAthletes.txt")
documents = loader.load()

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

In [9]:
retriever = db.as_retriever()



In [10]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "skateboarding_athletes",
    "Searches for skateboarding athletes in Paris 2024 in 4 variations, Men(Park), Women(Park), Men(Street), Women(Street)",
)
tools = [tool]

In [11]:
from langchain.agents import AgentExecutor, create_openai_tools_agent


In [12]:
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [25]:
question = "Rules of Street Skateboarding."

In [26]:
result = agent_executor.invoke({"input": question})
result2 = llm.invoke(question)




In [27]:
result

{'input': 'Rules of Street Skateboarding.',
 'output': "Street skateboarding is a popular and dynamic sport that involves performing tricks and maneuvers on urban obstacles such as stairs, rails, ledges, and ramps. Here are some key rules and guidelines for street skateboarding:\n\n1. **Respect for Public Property**: Skaters should always respect public and private property. Skateboarding should only be done in designated skate parks or areas where skateboarding is permitted.\n\n2. **Safety Gear**: It is highly recommended to wear safety gear such as a helmet, knee pads, elbow pads, and wrist guards to prevent injuries while skateboarding.\n\n3. **Right of Way**: Skaters should be aware of their surroundings and give the right of way to pedestrians and other skaters. It is important to avoid collisions and be mindful of others sharing the space.\n\n4. **Trick Execution**: Street skateboarding involves performing a variety of tricks such as ollies, kickflips, grinds, and slides on obsta

In [28]:
result2

AIMessage(content='1. Always wear protective gear, including a helmet, knee pads, elbow pads, and wrist guards.\n\n2. Skate in designated skateboarding areas, such as skate parks or designated skateboarding zones.\n\n3. Be aware of your surroundings and watch out for pedestrians, cyclists, and other obstacles.\n\n4. Respect other skaters and share the space with them.\n\n5. Do not skate on private property without permission.\n\n6. Do not skate on public property that is not designated for skateboarding.\n\n7. Do not skate under the influence of drugs or alcohol.\n\n8. Do not perform tricks that are beyond your skill level, as this can lead to injury.\n\n9. Always warm up before skating to prevent injuries.\n\n10. Have fun and enjoy the sport responsibly.', response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 13, 'total_tokens': 166}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-78649983-353